In [2]:
import pandas as pd
import json
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Исследуем датасет (валидационный, из-за размера)

In [ ]:
def read_json(data_path, dataset_type):
    with open(data_path+f"{dataset_type}.json", "rb") as f:
        json_object = json.load(f)

    return json_object

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Хакатон полы/data/val/"
dataset_type = "val"
val = read_json(data_path, dataset_type)

In [ ]:
df = pd.DataFrame(val)
df.head()

,user_127756,user_127757,user_127758,user_127759,user_127760,user_127761,user_127762,user_127763,user_127764,user_127765,...,user_155193,user_155194,user_155195,user_155196,user_155197,user_155198,user_155199,user_155200,user_155201,user_155202
target,female,male,female,female,female,female,female,male,male,female,...,male,female,male,female,male,female,male,female,male,female
features,"{'orders': [{'site-id': 407, 'orders': [{'crea...","{'orders': [{'site-id': 16, 'orders': [{'creat...","{'orders': [{'site-id': 149, 'orders': [{'crea...","{'orders': [{'site-id': 93, 'orders': [{'creat...","{'visits': [{'site-id': 3, 'first-seen': 16836...","{'orders': [{'site-id': 62, 'orders': [{'creat...","{'orders': [{'site-id': 214, 'orders': [{'crea...","{'orders': [{'site-id': 489, 'orders': [{'crea...","{'orders': [{'site-id': 179, 'orders': [{'crea...","{'orders': [{'site-id': 16, 'orders': [{'creat...",...,"{'orders': [{'site-id': 8, 'orders': [{'create...","{'orders': [{'site-id': 2, 'orders': [{'create...","{'orders': [{'site-id': 54, 'orders': [{'creat...","{'orders': [{'site-id': 39, 'orders': [{'creat...","{'orders': [{'site-id': 40, 'orders': [{'creat...","{'orders': [{'site-id': 7, 'orders': [{'create...","{'orders': [{'site-id': 21, 'orders': [{'creat...","{'orders': [{'site-id': 34, 'orders': [{'creat...","{'orders': [{'site-id': 218, 'orders': [{'crea...","{'orders': [{'site-id': 49, 'orders': [{'creat..."


In [ ]:
df.loc['features'][1]

{'orders': [{'site-id': 16,
   'orders': [{'created-at': 1636574129,
     'items': [{'id': 'item_3780263',
       'count': 2,
       'general-category-path': [90578, 90574, 198118],
       'brand-id': 1237}]},
    {'created-at': 1637512604,
     'items': [{'id': 'item_5403882',
       'count': 2,
       'general-category-path': [90578, 90574, 198118],
       'brand-id': 1237}]},
    {'created-at': 1637948102,
     'items': [{'id': 'item_1627033',
       'count': 1,
       'general-category-path': [91498, 91497, 91461, 198119],
       'brand-id': 1}]},
    {'created-at': 1638631112,
     'items': [{'id': 'item_554781',
       'count': 1,
       'general-category-path': [90629, 90625, 10599873, 198119],
       'brand-id': 3519}]},
    {'created-at': 1641841471,
     'items': [{'id': 'item_1898950',
       'count': 1,
       'general-category-path': [90578, 90574, 198118],
       'brand-id': 996}]},
    {'created-at': 1644774543,
     'items': [{'id': 'item_2110277',
       'count': 1,
  

# Построение модели по товарным категориям

In [ ]:
def parse_orders(json_object):
    # parse json
    df = pd.DataFrame()
    for j in range(len(list(json_object.keys()))):
        key = list(json_object.keys())[j]
        if "orders" in json_object[key]["features"].keys():
            dfl2 = pd.DataFrame()
            for n in range(len(json_object[key]["features"]["orders"])):
                l = json_object[key]["features"]["orders"][n]
                dfl1 = pd.DataFrame()
                for i in range(len(l["orders"])):
                    k = l["orders"][i]
                    dfl0 = pd.DataFrame(k["items"])
                    dfl0["created-at"] = k["created-at"]

                    for j in range(len(k["items"])):
                        m = k["items"][j]
                        dfl00 = pd.DataFrame(m.get("general-category-path"))
                        dfl0 = pd.concat([dfl0, dfl00]).reset_index(drop=True)

                    dfl1 = pd.concat([dfl1, dfl0]).reset_index(drop=True)
                dfl1["site-id"] = l["site-id"]
                dfl2 = pd.concat([dfl2, dfl1]).reset_index(drop=True)
                dfl2["user"] = key
                if "target" in json_object[key]:
                    dfl2["target"] = json_object[key]["target"]
            df = pd.concat([df, dfl2])

    return df

In [ ]:
general_category_path = parse_orders(val)

In [ ]:
general_category_path.sample(5)

,id,count,created-at,site-id,user,target,general-category-path,brand-id,0
16,NaN,NaN,NaN,182,user_148116,female,NaN,NaN,7812192.0
11,item_16984,1.0,1.445687e+09,54,user_148744,female,NaN,NaN,NaN
47,NaN,NaN,NaN,173,user_136637,female,NaN,NaN,12327586.0
223,NaN,NaN,NaN,77,user_153458,female,NaN,NaN,13314794.0
3,item_335290,1.0,1.587717e+09,50,user_148895,female,NaN,NaN,NaN


In [140]:
cat_dump_path = '/content/drive/MyDrive/Colab Notebooks/Хакатон полы/general_category_path.csv'
#general_category_path.to_csv(cat_dump_path, index=False)

In [141]:
general_category_path = pd.read_csv(cat_dump_path)
general_category_path

,id,count,created-at,site-id,user,target,general-category-path,brand-id,0
0,item_676374,NaN,1.697126e+09,407,user_127756,female,NaN,NaN,NaN
1,item_3780263,2.0,1.636574e+09,16,user_127757,male,"[90578, 90574, 198118]",1237.0,NaN
2,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,90578.0
3,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,90574.0
4,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,198118.0
...,...,...,...,...,...,...,...,...,...
392005,NaN,NaN,NaN,21,user_133185,male,NaN,NaN,281934.0
392006,NaN,NaN,NaN,21,user_133185,male,NaN,NaN,281933.0
392007,NaN,NaN,NaN,21,user_133185,male,NaN,NaN,91763.0
392008,item_959646,1.0,1.506631e+09,151,user_133185,male,NaN,NaN,NaN


Существует или general-category-path, или колонка 0.
Далее строим модель по колонке 0, которую переименуем в order-category

## Модель по visits-order-category и распределению полов - 74%

In [142]:
general_category_path.rename(columns={'0': 'order-category'}, inplace=True)
general_category_useful = general_category_path.dropna(subset = ['order-category'])
general_category_useful

,id,count,created-at,site-id,user,target,general-category-path,brand-id,order-category
2,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,90578.0
3,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,90574.0
4,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,198118.0
6,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,90578.0
7,NaN,NaN,NaN,16,user_127757,male,NaN,NaN,90574.0
...,...,...,...,...,...,...,...,...,...
392002,NaN,NaN,NaN,98,user_133185,male,NaN,NaN,6179129.0
392003,NaN,NaN,NaN,98,user_133185,male,NaN,NaN,90666.0
392005,NaN,NaN,NaN,21,user_133185,male,NaN,NaN,281934.0
392006,NaN,NaN,NaN,21,user_133185,male,NaN,NaN,281933.0


In [143]:
general_category_final = general_category_useful.drop(['id', 'count', 'created-at', 'general-category-path', 'brand-id'], axis=1)
general_category_final['order-category'] = general_category_final['order-category'].astype('int').astype('str')
general_category_final

,site-id,user,target,order-category
2,16,user_127757,male,90578
3,16,user_127757,male,90574
4,16,user_127757,male,198118
6,16,user_127757,male,90578
7,16,user_127757,male,90574
...,...,...,...,...
392002,98,user_133185,male,6179129
392003,98,user_133185,male,90666
392005,21,user_133185,male,281934
392006,21,user_133185,male,281933


In [144]:
# Функция замены order-category на унифицированный sex_score

def replace_order_category(df):
    # Переводим пол в бинарный признак через One-Hot Encoding
    df_encoded = pd.get_dummies(df, columns=['target'], prefix='target')
    # Группируем по сайту, считаем sex.score (уровень феминности) для каждого сайта
    df_grouped = df_encoded.groupby('order-category').agg({'user':'count', 'target_female':'sum'}).reset_index()
    df_grouped['target_female'] = df_grouped['target_female'] / df_grouped['user']
    # Переводим sex.scores в словарь
    df_grouped.set_index('order-category', drop=True, inplace=True)
    sex_score_dic = df_grouped['target_female'].to_dict()
    # Заменяем order-category на sex.score
    df['order-category'] = df['order-category'].replace(sex_score_dic)
    df = df.groupby('user').agg({'order-category': 'mean', 'target': 'last'}).reset_index()
    return df

In [145]:
replaced_order_category = replace_order_category(general_category_final)
replaced_order_category.sample()

,user,order-category,target
2996,user_132620,0.644373,female


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = replaced_order_category[['order-category']]
y = replaced_order_category['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Accuracy: 0.74
Confusion Matrix:
[[323  47]
 [127 172]]
Classification Report:
              precision    recall  f1-score   support

      female       0.72      0.87      0.79       370
        male       0.79      0.58      0.66       299

    accuracy                           0.74       669
   macro avg       0.75      0.72      0.73       669
weighted avg       0.75      0.74      0.73       669



## Модель по order-site-id и распределению полов - 74%

In [146]:
# Функция замены site_id на унифицированный sex_score

def replace_site_id(df):
    # Переводим пол в бинарный признак через One-Hot Encoding
    df_encoded = pd.get_dummies(df, columns=['target'], prefix='target')
    # Группируем по сайту, считаем sex.score (уровень феминности) для каждого сайта
    df_grouped = df_encoded.groupby('site-id').agg({'user':'count', 'target_female':'sum'}).reset_index()
    df_grouped['target_female'] = df_grouped['target_female'] / df_grouped['user']
    # Переводим sex.scores в словарь
    df_grouped.set_index('site-id', drop=True, inplace=True)
    sex_score_dic = df_grouped['target_female'].to_dict()
    # Заменяем site_id на sex.score
    df['site-id'] = df['site-id'].replace(sex_score_dic)
    df = df.groupby('user').agg({'site-id': 'mean', 'target': 'last'}).reset_index()
    return df

In [147]:
replaced_site_id = replace_site_id(general_category_final)
replaced_site_id

,user,site-id,target
0,user_127757,0.358513,male
1,user_127758,0.719134,female
2,user_127759,0.897634,female
3,user_127761,0.678631,female
4,user_127763,0.431816,male
...,...,...,...
3338,user_133179,0.637198,male
3339,user_133180,0.483599,male
3340,user_133181,0.916660,female
3341,user_133182,0.940747,female


In [ ]:
X = replaced_order_category[['order-category']]
y = replaced_order_category['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Accuracy: 0.74
Confusion Matrix:
[[323  47]
 [127 172]]
Classification Report:
              precision    recall  f1-score   support

      female       0.72      0.87      0.79       370
        male       0.79      0.58      0.66       299

    accuracy                           0.74       669
   macro avg       0.75      0.72      0.73       669
weighted avg       0.75      0.74      0.73       669



## Объединенная модель по order-category, order-site-id и распределению полов - 76%

In [148]:
merged_df = replaced_order_category.merge(replaced_site_id[['user', 'site-id']], on='user', how='left')

In [149]:
merged_df = merged_df.dropna()

In [150]:
merged_df

,user,order-category,target,site-id
0,user_127757,0.459163,male,0.358513
1,user_127758,0.725644,female,0.719134
2,user_127759,0.818264,female,0.897634
3,user_127761,0.573988,female,0.678631
4,user_127763,0.469878,male,0.431816
...,...,...,...,...
3338,user_133179,0.612359,male,0.637198
3339,user_133180,0.412703,male,0.483599
3340,user_133181,0.655548,female,0.916660
3341,user_133182,0.734463,female,0.940747


In [ ]:
X = merged_df[['order-category', 'site-id']]
y = merged_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Accuracy: 0.76
Confusion Matrix:
[[323  47]
 [114 185]]
Classification Report:
              precision    recall  f1-score   support

      female       0.74      0.87      0.80       370
        male       0.80      0.62      0.70       299

    accuracy                           0.76       669
   macro avg       0.77      0.75      0.75       669
weighted avg       0.77      0.76      0.75       669



# Объединение товарных категорий с метаданными сайтов

## Модель по site-meta, site-id и распределению полов

In [ ]:
# Функция собирает все ["site-meta"]['site-id']

def parse_site_meta_ids(json_object):

    # Создаем пустой df
    df = pd.DataFrame()
    # Проходим по ключам словаря (users)
    for j in range(len(list(json_object.keys()))):
        key = list(json_object.keys())[j]
        # Создаем пустой df для итерации по юзерам
        dfl1 = pd.DataFrame()
        if "site-meta" in json_object[key]["features"]:
            dfl2 = pd.json_normalize(json_object[key]["features"]["site-meta"])['site-id']
            dfl1 = pd.concat([dfl1, dfl2]).reset_index(drop=True)
        # Добавляем ключ (user)
        dfl1["user"] = [key] * len(dfl1)
        # Добавляем пол (target)
        if "target" in json_object[key].keys():
            dfl1["target"] = json_object[key]["target"]
        # Пришиваем полученный df к итоговому
        df = pd.concat([df, dfl1]).reset_index(drop=True)

    return df

In [ ]:
meta_path = '/content/drive/MyDrive/Colab Notebooks/Хакатон полы/meta.csv'

meta_ids = pd.read_csv(meta_path)
meta_ids

,0,user,target
0,407.0,user_127756,female
1,16.0,user_127757,male
2,29.0,user_127757,male
3,21.0,user_127757,male
4,40.0,user_127757,male
...,...,...,...
302020,76.0,user_155202,female
302021,3.0,user_155202,female
302022,603.0,user_155202,female
302023,86.0,user_155202,female


In [ ]:
# Функция замены site_id на унифицированный sex_score

def replace_site_id(df):
    # Переводим пол в бинарный признак через One-Hot Encoding
    df_encoded = pd.get_dummies(df, columns=['target'], prefix='target')
    # Группируем по сайту, считаем sex.score (уровень феминности) для каждого сайта
    df_grouped = df_encoded.groupby(0).agg({'user':'count', 'target_female':'sum'}).reset_index()
    df_grouped['target_female'] = df_grouped['target_female'] / df_grouped['user']
    # Переводим sex.scores в словарь
    df_grouped.set_index(0, drop=True, inplace=True)
    sex_score_dic = df_grouped['target_female'].to_dict()
    # Заменяем site_id на sex.score
    df[0] = df[0].replace(sex_score_dic)
    df = df.groupby('user').agg({0: 'mean', 'target': 'last'}).reset_index()
    return df

In [ ]:
replaced_meta_df = replace_site_id(meta_ids)

In [ ]:
replaced_meta_df.rename(columns={0:'meta_site_id'}, inplace=True)
replaced_meta_df

,user,meta_site_id,target
0,user_127756,0.798419,female
1,user_127757,0.478625,male
2,user_127758,0.611255,female
3,user_127759,0.593791,female
4,user_127760,0.740496,female
...,...,...,...
27442,user_155198,0.548838,female
27443,user_155199,0.525881,male
27444,user_155200,0.610900,female
27445,user_155201,0.537106,male


In [ ]:
X = replaced_meta_df[['meta_site_id']]
y = replaced_meta_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Accuracy: 0.80
Confusion Matrix:
[[2216  566]
 [ 557 2151]]
Classification Report:
              precision    recall  f1-score   support

      female       0.80      0.80      0.80      2782
        male       0.79      0.79      0.79      2708

    accuracy                           0.80      5490
   macro avg       0.80      0.80      0.80      5490
weighted avg       0.80      0.80      0.80      5490



## Объединение по order-category, order-site-id, meta-site-id и распределению полов - нет

In [ ]:
merged_df_1 = replaced_meta_df.merge(merged_df[['user', 'order-category', 'site-id']], on='user', how='left')

In [ ]:
merged_df_1 = merged_df.dropna()

In [ ]:
merged_df_1

,user,order-category,target,site-id
0,user_127757,0.459163,male,0.358513
1,user_127758,0.725644,female,0.719134
2,user_127759,0.818264,female,0.897634
3,user_127761,0.573988,female,0.678631
4,user_127763,0.469878,male,0.431816
...,...,...,...,...
3338,user_133179,0.612359,male,0.637198
3339,user_133180,0.412703,male,0.483599
3340,user_133181,0.655548,female,0.916660
3341,user_133182,0.734463,female,0.940747


Видим, что признаки из группы site-meta и признаки группы orders не пересекаются, объединение моделей сделать невозможно

# Модель по visits-general-category-path со словарем general-categories-mapping - 68%

In [8]:
dct_path = '/content/drive/MyDrive/Colab Notebooks/Хакатон полы/general_categories_mapping_corr.json'

In [126]:
with open(dct_path, 'rb') as file:
    cat_dict = json.load(file)
cat_dict[:10]

[{'1016160': 'Воланы', 'guess': 'fa'},
 {'16065434': 'Катетеры урологические', 'guess': 'fa'},
 {'15450095': 'Винные шкафы', 'guess': 'ma'},
 {'91230': 'Pop', 'guess': 'fa'},
 {'819169': 'Корректоры метаболизма хрящевой ткани', 'guess': 'ma'},
 {'294642': 'Чейнджеры', 'guess': 'ma'},
 {'15259392': 'Плазменная резка', 'guess': 'ma'},
 {'90907': 'Нормативные акты и документы (inactive)', 'guess': 'fa'},
 {'90921': 'Антикварные и редкие книги (inactive)', 'guess': 'fa'},
 {'6856242': 'Зарядные устройства для аккумуляторов', 'guess': 'ma'}]

In [127]:
# замена значений на числа в словаре
result_cat_dict = {}
for item in cat_dict:
    key = list(item.keys())[0]
    key = str(key)
    value = item['guess']
    if value == 'fe':
      value = float(1)
    elif value == 'fa':
      value = float(0.5)
    elif value == 'ma':
      value = float(0)
    result_cat_dict[key] = value
result_cat_dict

{'1016160': 0.5,
 '16065434': 0.5,
 '15450095': 0.0,
 '91230': 0.5,
 '819169': 0.0,
 '294642': 0.0,
 '15259392': 0.0,
 '90907': 0.5,
 '90921': 0.5,
 '6856242': 0.0,
 '961226': 0.5,
 '418700': 0.0,
 '7962984': 0.5,
 '14213973': 1.0,
 '15093378': 1.0,
 '6391089': 0.0,
 '14369211': 0.0,
 '15085725': 0.5,
 '15775109': 0.0,
 '947515': 0.0,
 '14732168': 0.0,
 '6368402': 0.5,
 '13062140': 0.0,
 '819421': 1.0,
 '819422': 1.0,
 '819423': 1.0,
 '2562535': 0.5,
 '14405299': 0.0,
 '14420937': 1.0,
 '91093': 0.0,
 '13793704': 1.0,
 '91726': 0.0,
 '90610': 0.5,
 '14912198': 0.0,
 '15694618': 0.0,
 '14419833': 0.0,
 '14193335': 1.0,
 '658844': 0.0,
 '90753': 1.0,
 '14700323': 0.0,
 '14815024': 0.0,
 '11031140': 0.0,
 '12327183': 0.0,
 '950505': 0.5,
 '14814983': 0.0,
 '91775': 0.5,
 '91772': 0.0,
 '15724921': 0.0,
 '91774': 0.0,
 '638257': 0.0,
 '15337351': 0.0,
 '142665': 0.0,
 '15694311': 0.0,
 '15157810': 0.0,
 '2134462': 0.0,
 '7797084': 0.5,
 '13525587': 0.0,
 '15294634': 0.5,
 '91687': 0.0,
 '1

In [128]:
general_category_path = general_category_path.dropna(subset = ['general-category-path'])
general_category_path

,id,count,created-at,site-id,user,target,general-category-path,brand-id,order-category
1,item_3780263,2.0,1.636574e+09,16,user_127757,male,"[90578, 90574, 198118]",1237.0,NaN
5,item_5403882,2.0,1.637513e+09,16,user_127757,male,"[90578, 90574, 198118]",1237.0,NaN
9,item_1627033,1.0,1.637948e+09,16,user_127757,male,"[91498, 91497, 91461, 198119]",1.0,NaN
14,item_554781,1.0,1.638631e+09,16,user_127757,male,"[90629, 90625, 10599873, 198119]",3519.0,NaN
19,item_1898950,1.0,1.641841e+09,16,user_127757,male,"[90578, 90574, 198118]",996.0,NaN
...,...,...,...,...,...,...,...,...,...
391975,item_1496681,1.0,1.574946e+09,389,user_133185,male,"[10498025, 91461, 198119]",2575.0,NaN
391979,item_1100018,1.0,1.610643e+09,389,user_133185,male,"[91491, 91461, 198119]",1.0,NaN
391994,item_830534,1.0,1.641243e+09,159,user_133185,male,"[818863, 15068776, 7811881, 7877999]",4516.0,NaN
391999,item_15007,1.0,1.636611e+09,98,user_133185,male,"[7286125, 90679, 6179129, 90666]",2441.0,NaN


In [129]:
general_category_useful = general_category_path.drop(['site-id', 'id', 'count', 'created-at', 'order-category', 'brand-id'], axis=1)
general_category_useful

,user,target,general-category-path
1,user_127757,male,"[90578, 90574, 198118]"
5,user_127757,male,"[90578, 90574, 198118]"
9,user_127757,male,"[91498, 91497, 91461, 198119]"
14,user_127757,male,"[90629, 90625, 10599873, 198119]"
19,user_127757,male,"[90578, 90574, 198118]"
...,...,...,...
391975,user_133185,male,"[10498025, 91461, 198119]"
391979,user_133185,male,"[91491, 91461, 198119]"
391994,user_133185,male,"[818863, 15068776, 7811881, 7877999]"
391999,user_133185,male,"[7286125, 90679, 6179129, 90666]"


In [130]:
# превращаем строку с категориями в список

def string_replace(x=None):
  row = x.replace('[', '').replace(']', '')
  row = row.split(', ')

  return row

general_category_useful['category'] = general_category_useful['general-category-path'].apply(string_replace)

In [131]:
# заменяем категории на числа, для отсутствующих категорий ставим 0.5

import numpy as np

def category_replace(x=None):
  ele = [result_cat_dict[i] if i in result_cat_dict else float(0.5) for i in x]
  return ele

general_category_useful['category_num'] = general_category_useful['category'].apply(category_replace)

In [132]:
general_category_useful.drop(['general-category-path'], axis=1, inplace = True)
general_category_useful

,user,target,category,category_num
1,user_127757,male,"[90578, 90574, 198118]","[0.0, 0.5, 0.0]"
5,user_127757,male,"[90578, 90574, 198118]","[0.0, 0.5, 0.0]"
9,user_127757,male,"[91498, 91497, 91461, 198119]","[0.5, 0.0, 0.5, 0.5]"
14,user_127757,male,"[90629, 90625, 10599873, 198119]","[0.0, 0.5, 0.5, 0.5]"
19,user_127757,male,"[90578, 90574, 198118]","[0.0, 0.5, 0.0]"
...,...,...,...,...
391975,user_133185,male,"[10498025, 91461, 198119]","[0.5, 0.5, 0.5]"
391979,user_133185,male,"[91491, 91461, 198119]","[0.5, 0.5, 0.5]"
391994,user_133185,male,"[818863, 15068776, 7811881, 7877999]","[0.5, 1.0, 0.5, 0.5]"
391999,user_133185,male,"[7286125, 90679, 6179129, 90666]","[0.5, 0.5, 0.0, 0.5]"


In [133]:
def category_sum(x=None):
  row_sum = 0
  for i in x:
    row_sum += i
  return row_sum

general_category_useful['category_sum'] = general_category_useful['category_num'].apply(category_sum)

In [134]:
def category_count(x=None):
  row_count = 0
  for i in x:
    row_count += 1
  return row_count

general_category_useful['category_count'] = general_category_useful['category_num'].apply(category_count)

In [135]:
general_category_short = general_category_useful.drop(['category', 'category_num'], axis=1)
general_category_short.columns

Index(['user', 'target', 'category_sum', 'category_count'], dtype='object')

In [136]:
general_category_short = general_category_short.groupby('user').agg({'category_sum': 'sum', 'category_count': 'sum', 'target': 'last'}).reset_index()
general_category_short

,user,category_sum,category_count,target
0,user_127757,15.5,39,male
1,user_127758,60.5,129,female
2,user_127759,69.5,88,female
3,user_127761,5.5,11,female
4,user_127763,4.5,9,male
...,...,...,...,...
3338,user_133179,12.0,30,male
3339,user_133180,3.5,15,male
3340,user_133181,22.0,54,female
3341,user_133182,41.0,86,female


In [137]:
general_category_short['category_score'] = general_category_short['category_sum']/general_category_short['category_count']
general_category_short['target'] = pd.factorize(general_category_short['target'])[0]
general_category_short

,user,category_sum,category_count,target,category_score
0,user_127757,15.5,39,0,0.397436
1,user_127758,60.5,129,1,0.468992
2,user_127759,69.5,88,1,0.789773
3,user_127761,5.5,11,1,0.500000
4,user_127763,4.5,9,0,0.500000
...,...,...,...,...,...
3338,user_133179,12.0,30,0,0.400000
3339,user_133180,3.5,15,0,0.233333
3340,user_133181,22.0,54,1,0.407407
3341,user_133182,41.0,86,1,0.476744


In [138]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = general_category_short[['category_score']]
y = general_category_short['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Accuracy: 0.68
Confusion Matrix:
[[151 148]
 [ 68 302]]
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.51      0.58       299
           1       0.67      0.82      0.74       370

    accuracy                           0.68       669
   macro avg       0.68      0.66      0.66       669
weighted avg       0.68      0.68      0.67       669



# Объединенная модель по visits-general-category, order-category, order-site-id, general-categories-mapping - 76%

In [151]:
merged_df_2 = merged_df.merge(general_category_short[['user', 'category_score']], on='user', how='left')

In [152]:
merged_df_2.dropna()

,user,order-category,target,site-id,category_score
0,user_127757,0.459163,male,0.358513,0.397436
1,user_127758,0.725644,female,0.719134,0.468992
2,user_127759,0.818264,female,0.897634,0.789773
3,user_127761,0.573988,female,0.678631,0.500000
4,user_127763,0.469878,male,0.431816,0.500000
...,...,...,...,...,...
3338,user_133179,0.612359,male,0.637198,0.400000
3339,user_133180,0.412703,male,0.483599,0.233333
3340,user_133181,0.655548,female,0.916660,0.407407
3341,user_133182,0.734463,female,0.940747,0.476744


In [153]:
X = merged_df_2[['order-category', 'site-id', 'category_score']]
y = merged_df_2['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Accuracy: 0.76
Confusion Matrix:
[[321  49]
 [109 190]]
Classification Report:
              precision    recall  f1-score   support

      female       0.75      0.87      0.80       370
        male       0.79      0.64      0.71       299

    accuracy                           0.76       669
   macro avg       0.77      0.75      0.75       669
weighted avg       0.77      0.76      0.76       669



Модель основана на правилах (меппинг словаря), показала результат не выше, чем модель с распределением по полам на основе текущего распределения order-site-id и visits-order-category